# Training

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  591530427655.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker images

## Build

In [ ]:
!docker images

In [ ]:
!docker build -t 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2 -f "/home/ec2-user/SageMaker/UFS14Forkata/docker/Dockerfile.train" /home/ec2-user/SageMaker/UFS14Forkata

In [ ]:
!cd ..; ls
print("########################################################")
!cd .; ls

## Esecuzione Locale

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os
import pandas as pd
role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!SM_MODEL_DIR="./data/output" python main.py

In [ ]:
!tar -xzf ./data/output/model.tar.gz

## Init repo on ECR

In [ ]:
%%sh
# Specify an image name
image_name=my-tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pippo-2"
echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"


In [ ]:
!docker images

In [ ]:
#!docker rmi tensorflow-training:2.12.0-cpu-py310-pippo-2

In [ ]:
#!docker tag tensorflow-training:2.12.0-cpu-py310-pippo-2 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2

In [ ]:
#!docker tag bec0922c4a51 tensorflow-training:2.12.0-cpu-py310-pippo-2

## Push image

In [ ]:
#!aws sagemaker describe-notebook-instance \
    #--notebook-instance-name CFNNotebookInstance-fBxJOmvnjTpG \
    #--region us-east-1

In [ ]:
#!aws iam attach-role-policy \
    #--role-name c133245a3375001l7613526t1w00-SageMakerExecutionRole-XxDt97bVnO4D \
    #--policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

In [ ]:
!docker push 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2

## Esecuzione su ECR

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os
import pandas as pd

role=get_execution_role()

MODEL_INPUT_PATH = 's3://marcorogicesame/data/input/user_behavior_dataset.csv'

MODEL_OUTPUT_PATH = 's3://marcorogicesame/data/output'

INSTANCE_TYPE = 'ml.p2.xlarge'

hyperparameters={'epochs': 50}

estimator=Estimator(
    image_uri='591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2',
    role=role,
    instance_count=1,
    instance_type=INSTANCE_TYPE,
    hyperparameters=hyperparameters,
    output_path=MODEL_OUTPUT_PATH
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit(MODEL_INPUT_PATH)
print('##### ESTIMATOR FIT COMPLETED')

# Build immagine inferenza

In [ ]:
!docker build -t my-flask-inference -f "/home/ec2-user/SageMaker/UFS14Forkata/docker/Dockerfile.inference" /home/ec2-user/SageMaker/UFS14Forkata

In [ ]:
!docker images

# Init ECR per immagine inferenza

In [ ]:
%%sh
# Specify an image name
image_name=my-flask-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:1.12.0-cpu"

echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 591530427655.dkr.ecr.us-east-1.amazonaws.com

# Push dell'immagine di inferenza all'interno dell'ECR

In [ ]:
!docker tag my-flask-inference:latest 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-flask-inference:latest

In [ ]:
!docker push 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-flask-inference:latest

# Prova inferenza in locale

In [ ]:
from keras import models
import numpy as np
import json

print(os.path.exists(f"/home/ec2-user/SageMaker/UFS14Forkata/model.keras"))
model = models.load_model("/home/ec2-user/SageMaker/UFS14Forkata/model.keras")
type(model)

predict_input = np.array([[0.475,0.37,0.125,0.5095,0.2165,0.1125,0.165,9]])
predict_result = model.predict(predict_input)

json.dumps({"predict_result": predict_result.tolist()})

In [ ]:
!docker images

# Creazione endpoint e deploy

In [ ]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'il-top'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '591530427655.dkr.ecr.us-east-1.amazonaws.com/my-flask-inference:latest', ##Inference URI in ECR
        'ModelDataUrl': 's3://brunobucket69/data/output/my-tensorflow-training-2024-11-21-20-56-58-105/output/model.tar.gz',
    })


In [ ]:
endpoint_config_name = 'boiadee'

instance_type = 'ml.p2.xlarge'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1", 
            "ModelName": model_name,
            "InstanceType": instance_type, 
            "InitialInstanceCount": 1 
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

endpoint_name = 'primo-endpoint-boia'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)

In [ ]:
import boto3
import json

runtime_client = boto3.client('sagemaker-runtime', region_name=aws_region)

payload = json.dumps({"instances": [[0.475, 0.37, 0.125, 0.5095, 0.2165, 0.1125, 0.165, 9]]})

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  
    Body=payload                     
)

result = json.loads(response['Body'].read().decode())
print(f"Inference result: {result}")
